In [1]:
import sys
sys.path.append('../')
from utils import paths
from utils import utils

import os
import warnings
from tqdm import tqdm
import pandas as pd
import numpy as np
import datetime
import xarray as xr
import time
from pyTMD import compute_tide_corrections

In [2]:

def geo_coord_borders(lat, lon):
    """
    Get the borders of the window of interest for CMIP6 Native grid;
    To be passed down into extract_window function.
    ---
    lat (np.float): latitude of location
    lon (np.float): longitude of location
    ---
    Returns:
        ((lat_lw_bord,lat_up,bord),(lon_lw_bord,lon_up_bord)): (tuple(tuple(np.float)))
    """
    ### The CMIP6 N96 Native grid has   144 gridcells of latitude, i.e. 1.25deg/cell, and
    ###                                 192 gridcells of longitude, i.e. 1.875deg/cell.

    ### Make sure the longitude is in [-180,180], not [0,360]:
    if (lon - 180 >0):
        lon = lon - 360

    delta_lat = 1.25 * 2.5
    delta_lon = 1.875* 2.5

    lat_lw_bord = lat - delta_lat 
    lat_up_bord = lat + delta_lat

    lon_lw_bord = lon - delta_lon
    lon_up_bord = lon + delta_lon

    if ((lon_lw_bord >= 0)&(lon_up_bord > 0)):
        lon_lw_bord = lon - delta_lon
        lon_up_bord = lon + delta_lon
    elif ((lon_lw_bord < 0)&(lon_up_bord <= 0)):
        lon_lw_bord = lon - delta_lon + 360
        lon_up_bord = lon + delta_lon + 360
    elif ((lon_lw_bord < 0)&(lon_up_bord > 0)):
        lon_lw_bord = lon - delta_lon + 360
        lon_up_bord = lon + delta_lon
    else:
        print("Error: Something is wrong with the Longitude.")

    return ((lat_lw_bord,lat_up_bord),(lon_lw_bord,lon_up_bord))

#--------------------------------------------------------------------------------------------------
def extract_window(data,geo_coord_tuple,data_type = 'CMIP6'):
    """
    data_type (string): 'CMIP6' or 'ERA5'
    THIS FUNCTION FAILS FOR NEGATIVE LONGITUDES ---- FIX!
    """


    if data_type == 'CMIP6':

        lat_lw = geo_coord_tuple[0][0]
        lat_up = geo_coord_tuple[0][1]
        lon_lw = geo_coord_tuple[1][0]
        lon_up = geo_coord_tuple[1][1]

        data = data.sel(lat = slice(lat_lw,lat_up))

        ### Check if the window crosses the meridian:
        if ((lon_lw >180)&(lon_lw<360))&((lon_up>0)&(lon_up<180)):
            crop1 = data.sel(lon = slice(lon_lw,360))
            crop2 = data.sel(lon = slice(0,lon_up))
            data = xr.concat([crop1,crop2],dim = 'lon')
        else:
            data = data.sel(lon = slice(lon_lw,lon_up))
    elif data_type == 'ERA5':
        ### ERA5 data has longitude in [-180,180]
        lat_lw = geo_coord_tuple[0][1]
        lat_up = geo_coord_tuple[0][0]
        lon_lw = geo_coord_tuple[1][0]
        lon_up = geo_coord_tuple[1][1]

        data = data.sel(latitude = slice(lat_lw,lat_up))

        ### Check if the window crosses the meridian:
        if ((lon_lw >180)&(lon_lw<360))&((lon_up>0)&(lon_up<180)):
            crop1 = data.sel(longitude = slice(lon_lw-360,360))
            crop2 = data.sel(longitude = slice(0,lon_up))
            data = xr.concat([crop1,crop2],dim = 'longitude')
        elif ((lon_lw >180)&(lon_lw<360))&((lon_up>180)&(lon_up<360)):
            data = data.sel(longitude = slice(lon_lw-360,lon_up-360))
        else:
            data = data.sel(longitude = slice(lon_lw,lon_up))
    else:
        print('data_type not recognized.')

    return data
#--------------------------------------------------------------------------------------------------

def geo_coord_borders_1GRID(lat, lon, Dataset_type):
    """
    Get the borders of the window of interest for CMIP6 Native grid;
    To be passed down into extract_window function.
    ---
    lat (np.float): latitude of location
    lon (np.float): longitude of location
    ---
    Returns:
        ((lat_lw_bord,lat_up,bord),(lon_lw_bord,lon_up_bord)): (tuple(tuple(np.float)))
    """
    ### The CMIP6 N96 Native grid has   144 gridcells of latitude, i.e. 1.25deg/cell, and
    ###                                 192 gridcells of longitude, i.e. 1.875deg/cell.

    ### The CMIP6 N216 Native grid has  324 gridcells of latitude, i.e. 0.5555deg/cell, and
    ###                                 432 gridcells of longitude, i.e. 0.8333deg/cell.

    ### The ERA5 Native grid has        721 (720) gridcells of latitude, i.e. 0.5555deg/cell, and
    ###                                 1440 gridcells of longitude, i.e. 0.8333deg/cell.

    ### Make sure the longitude is in [-180,180], not [0,360]:
    if (lon - 180 >0):
        lon = lon - 360
    if Dataset_type == 'GCM_N216':
        delta_lat = 0.56 * .5
        delta_lon = 0.8333 * .5
    elif Dataset_type == 'ERA5':
        delta_lat = 0.25 * .5
        delta_lon = 0.25 * .5


    lat_lw_bord = lat - delta_lat 
    lat_up_bord = lat + delta_lat

    lon_lw_bord = lon - delta_lon
    lon_up_bord = lon + delta_lon

    if ((lon_lw_bord >= 0)&(lon_up_bord > 0)):
        lon_lw_bord = lon - delta_lon
        lon_up_bord = lon + delta_lon
    elif ((lon_lw_bord < 0)&(lon_up_bord <= 0)):
        lon_lw_bord = lon - delta_lon + 360
        lon_up_bord = lon + delta_lon + 360
    elif ((lon_lw_bord < 0)&(lon_up_bord > 0)):
        lon_lw_bord = lon - delta_lon + 360
        lon_up_bord = lon + delta_lon
    else:
        print("Error: Something is wrong with the Longitude.")

    return ((lat_lw_bord,lat_up_bord),(lon_lw_bord,lon_up_bord))


In [5]:
uk_haigh_meta = pd.read_csv(paths.uk_haigh_meta_path)
storms = pd.read_csv(paths.storms_path, skiprows = 2, header = 3)
water_level= pd.read_csv(paths.water_levels_path,skiprows = 2, header =3)

In [3]:
location = 'aberdeen'
# lat, lon = utils.GetCoordinates(location)
# flood_records = utils.ExtractFloods(location)
# geo_borders = utils.GetGeoBorders(lat,lon)

In [5]:
def GetFormattedData(location):
    start = time.time()
    warnings.filterwarnings("ignore")
    ps_data, u_data, v_data, t_data, pr_data = utils.Step1()
    ps_data, u_data, v_data, t_data, pr_data = utils.Step2(ps_data, u_data, v_data, t_data, pr_data)
    ps_window, u_window, v_window, t_window, pr_window = utils.Step3(ps_data, u_data, v_data, t_data, pr_data, location)
    df_ps, df_u, df_v, df_t, df_pr = utils.Step4(ps_window, u_window, v_window, t_window, pr_window)
    df_ps, df_u, df_v, df_t, df_pr = utils.Step5(df_ps, df_u, df_v, df_t, df_pr)
    df_ps, df_u, df_v, df_t, df_pr = utils.Step6(df_ps, df_u, df_v, df_t, df_pr)
    df_merged = utils.Step7(df_ps, df_u, df_v, df_t, df_pr)
    df_merged = utils.Step8(df_merged)
    df_merged = utils.Step9(df_merged)
    df_merged = utils.Step10(df_merged)
    df_merged = utils.Step11(df_merged)
    df_merged = utils.Step12(df_merged, location)
    df_merged = utils.Step13(df_merged, location)
    end = time.time()
    print("Time taken to get the data: ", (end-start)/60, ' mins.')
    return df_merged



In [6]:
final_df_aberdeen = GetFormattedData('aberdeen')

Time taken to open datasets:  0.03733563423156738  ; Step 1/13
Time taken to remove leap years:  0.030103206634521484  ; Step 2/13
Time taken to extract window:  0.0028951168060302734  ; Step 3/13
Time taken to convert to dataframe:  96.62711191177368  ; Step 4/13
Time taken to convert to fractional years:  82.65430235862732  ; Step 5/13
Time taken to drop time column:  2.238919973373413  ; Step 6/13
Time taken to merge dataframes:  105.1542980670929  ; Step 7/13


100%|██████████| 25/25 [03:59<00:00,  9.56s/it]


Time taken to create the grid-cell-wise features:  241.87705397605896  ; Step 8/13


100%|██████████| 24/24 [00:00<00:00, 100.15it/s]


Time taken to calculate vertical vorticity:  0.24085116386413574  ; Step 9/13


100%|██████████| 25/25 [00:16<00:00,  1.53it/s]


Time taken to calculate cumulative precipitation:  16.338603019714355  ; Step 10/13
Time taken to add NAO data:  1.3988449573516846  ; Step 11/13
Time taken to add tides:  0.07927703857421875  ; Step 12/13
Time taken to add floods:  0.7438199520111084 Step 13/13
Time taken to get the data:  9.126602919896444  mins.


In [7]:
final_df_aberdeen

,u_1_1,t,v_1_1,T_1_1,P_1_1,pr_1_1,u_1_2,v_1_2,T_1_2,P_1_2,...,pr_cum5_25_23,pr_cum3_25_24,pr_cum5_25_24,pr_cum3_25_25,pr_cum5_25_25,nao,time,tide,floods,floods_x4
0,-4.287253,1979.00000,-6.412828,272.940887,101650.906250,2.506189e-05,-4.466002,-6.803702,272.587158,101359.156250,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.874547,1979-01-01 00:00:00,0.141355,0.0,0.0
1,-2.441757,1979.00034,-5.705153,273.148224,101902.937500,1.115724e-06,-2.774396,-5.909136,272.749207,101616.664062,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.859784,1979-01-01 03:00:00,1.938312,0.0,0.0
2,0.916595,1979.00068,-4.984941,273.543762,102043.328125,1.115724e-06,0.338916,-5.134226,273.022766,101759.109375,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.772936,1979-01-01 06:00:00,0.031803,0.0,0.0
3,5.835155,1979.00103,-0.179356,274.211121,102134.421875,2.172031e-05,5.710859,-0.468808,274.435883,101840.609375,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.701229,1979-01-01 09:00:00,-1.895495,0.0,0.0
4,9.897849,1979.00137,0.128329,274.754761,102215.234375,5.587935e-07,10.333476,-0.077934,275.145081,101920.054688,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.538234,1979-01-01 12:00:00,0.110635,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116795,6.373770,2018.99829,5.895715,282.783936,103324.718750,3.285892e-05,7.295926,5.949275,282.698547,103061.726562,...,0.006905,0.004801,0.006398,0.004726,0.005789,0.791289,2018-12-31 09:00:00,-0.168054,0.0,0.0
116796,6.770332,2018.99863,4.397175,282.925079,103280.882812,4.455447e-05,7.543334,4.508853,282.785675,103017.890625,...,0.007051,0.004131,0.006581,0.004317,0.006006,1.361615,2018-12-31 12:00:00,1.570374,0.0,0.0
116797,7.294742,2018.99897,3.880948,283.198639,103248.695312,4.678033e-05,8.143503,3.962997,283.031372,102978.859375,...,0.006658,0.004452,0.006388,0.004548,0.006085,1.880161,2018-12-31 15:00:00,0.384296,0.0,0.0
116798,7.764698,2018.99932,3.455886,283.313660,103246.640625,6.685033e-06,8.889278,3.495771,283.172516,102969.265625,...,0.006172,0.004452,0.005975,0.004548,0.005762,2.217394,2018-12-31 18:00:00,-1.496255,0.0,0.0


In [11]:
final_df_aberdeen.to_csv('./final_df_' + 'aberdeen' + '.csv')


In [28]:
abr = pd.read_csv('/Users/tudor/Documents/phd/coding/project_CF/data/final_df_aberdeen.csv')

In [3]:
abr_tides = pd.read_csv('./tides_aberdeen.csv')

In [6]:
abr_tides['tide'].isnull().any()

False

In [10]:
abr['tide']

True

In [9]:
abr_tides = abr_tides

0         0.141355
1         1.938312
2         0.031803
3        -1.895495
4         0.110635
            ...   
116875    1.027660
116876    0.065842
116877   -0.920172
116878    0.047663
116879    1.198686
Name: tide, Length: 116880, dtype: float64

In [12]:

abr_tides['time_ok'] = pd.to_datetime(abr_tides['time'], format = '%Y-%m-%d %H:%M:%S')
abr_tides = abr_tides[~((abr_tides.time_ok.dt.month == 2) & (abr_tides.time_ok.dt.day == 29))]

In [15]:
abr_tides['tide'].isnull().any()

False

In [17]:
abr = utils.Step12(abr, 'aberdeen')

Time taken to add tides:  0.7802910804748535  ; Step 12/13


In [22]:
abr[abr['tide'].isnull()]

,Unnamed: 0,u_1_1,t,v_1_1,T_1_1,P_1_1,pr_1_1,u_1_2,v_1_2,T_1_2,...,pr_cum5_25_23,pr_cum3_25_24,pr_cum5_25_24,pr_cum3_25_25,pr_cum5_25_25,nao,time,tide,floods,floods_x4
3392,3392,6.167794,1980.16164,-1.524054,281.56076,102878.190,2.283417e-05,6.876872,-1.655105,281.37082,...,0.001313,0.000833,0.001467,0.000916,0.001642,0.919444,1980-03-01 00:00:00,NaN,0.0,0.0
3393,3393,5.594850,1980.16199,-3.708618,281.50150,102807.640,2.004951e-05,6.325235,-4.074421,281.26453,...,0.001268,0.000852,0.001454,0.000953,0.001661,1.093621,1980-03-01 03:00:00,NaN,0.0,0.0
3394,3394,2.796420,1980.16233,-7.893135,280.20862,102857.640,1.115724e-06,3.306624,-8.196262,279.98210,...,0.001265,0.000821,0.001454,0.000915,0.001665,1.265883,1980-03-01 06:00:00,NaN,0.0,0.0
3395,3395,2.500478,1980.16267,-6.821936,280.29922,102917.910,1.862645e-09,3.058033,-7.082898,280.15634,...,0.001250,0.000800,0.001439,0.000899,0.001650,1.219096,1980-03-01 09:00:00,NaN,0.0,0.0
3396,3396,3.050931,1980.16301,-6.121097,280.67210,102914.484,1.115724e-06,3.774213,-6.277219,280.47696,...,0.001268,0.000844,0.001473,0.000960,0.001701,1.199893,1980-03-01 12:00:00,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108587,108587,-1.404776,2016.18733,-3.085271,279.50467,102180.990,1.862645e-09,-1.815543,-3.288115,278.97147,...,0.005028,0.002669,0.005092,0.002777,0.005053,-2.141049,2016-03-10 09:00:00,NaN,0.0,0.0
108588,108588,-0.372529,2016.18767,-0.868799,279.73468,102339.195,1.862645e-09,-0.505111,-1.253975,279.21020,...,0.004940,0.002680,0.005034,0.002807,0.005025,-2.147823,2016-03-10 12:00:00,NaN,0.0,0.0
108589,108589,-1.404776,2016.18801,0.571624,279.88104,102357.690,1.862645e-09,-1.088709,0.157958,279.41928,...,0.004730,0.002680,0.004889,0.002807,0.004972,-1.989917,2016-03-10 15:00:00,NaN,0.0,0.0
108590,108590,-0.300319,2016.18835,2.761886,280.13544,102462.470,1.862645e-09,0.435985,2.038256,279.83746,...,0.004717,0.002680,0.004879,0.002807,0.004968,-1.710334,2016-03-10 18:00:00,NaN,0.0,0.0


In [29]:
abr['tide'] = abr_tides['tide'].to_list()
#  = tide_list

In [30]:
abr[abr['tide'].isnull()]

,Unnamed: 0,u_1_1,t,v_1_1,T_1_1,P_1_1,pr_1_1,u_1_2,v_1_2,T_1_2,...,pr_cum5_25_23,pr_cum3_25_24,pr_cum5_25_24,pr_cum3_25_25,pr_cum5_25_25,nao,time,tide,floods,floods_x4


In [25]:
abr_tides

,Unnamed: 0,lat,lon,time,tide,time_ok
0,0,57.14325,357.92549,1979-01-01 00:00:00,0.141355,1979-01-01 00:00:00
1,1,57.14325,357.92549,1979-01-01 03:00:00,1.938312,1979-01-01 03:00:00
2,2,57.14325,357.92549,1979-01-01 06:00:00,0.031803,1979-01-01 06:00:00
3,3,57.14325,357.92549,1979-01-01 09:00:00,-1.895495,1979-01-01 09:00:00
4,4,57.14325,357.92549,1979-01-01 12:00:00,0.110635,1979-01-01 12:00:00
...,...,...,...,...,...,...
116875,116875,57.14325,357.92549,2018-12-31 09:00:00,1.027660,2018-12-31 09:00:00
116876,116876,57.14325,357.92549,2018-12-31 12:00:00,0.065842,2018-12-31 12:00:00
116877,116877,57.14325,357.92549,2018-12-31 15:00:00,-0.920172,2018-12-31 15:00:00
116878,116878,57.14325,357.92549,2018-12-31 18:00:00,0.047663,2018-12-31 18:00:00


In [31]:
dover = pd.read_csv('./final_df_dover.csv')

In [34]:
dover_tide = pd.read_csv('./tides_dover.csv')

In [37]:
dover['tide'].isnull().any()
dover_tide.drop(columns = ['Unnamed: 0','lat','lon'] ,inplace = True)

In [43]:
dover.drop(columns = ['tide'], inplace = True)

In [48]:
dover_merged = pd.merge(dover, dover_tide, on='time', how = 'inner')


In [49]:
dover_merged['tide'].isnull().any()

False